# ge_scraper

- Takes data from the offical runescape GE api and extracts price of an item at a certain timestamp for the past 180 days. 
- The timestamps are taken for each day at 20:00:00. The untis are in ms

In [2]:
import requests
import json
import csv

In [6]:
'''Convert ms time from API to local time'''
import datetime

# Given timestamp in milliseconds
timestamps = [1729036800000, 1729123200000]

for ts in timestamps:
    # Convert to seconds
    ts /= 1000
    # Convert to a readable datetime (local time)
    readable_time = datetime.datetime.fromtimestamp(ts)
    print(f"Local Time: {ts} -> {readable_time}")

Local Time: 1729036800.0 -> 2024-10-15 20:00:00
Local Time: 1729123200.0 -> 2024-10-16 20:00:00


In [4]:
csvName = 'Rune_Data'

# get more item codes here: https://everythingrs.com/tools/osrs/itemlist/238
# itemList = [1521, 1519, 1517, 1515]  # Logs
itemList = list(range(554,567,1))  # Runes

fullDict = {}
labels = ['timestamp']


In [5]:
'''This grabs price at daily interval'''
# Construct dictionary full of data
for itemID in itemList:
    r = requests.get(f'http://services.runescape.com/m=itemdb_oldschool/api/graph/{itemID}.json')
    json_data = json.loads(r.text)
    current_daily_dict = json_data['daily']

    for daily_timestamp in current_daily_dict:
        if (daily_timestamp in fullDict):
            fullDict[daily_timestamp].append(current_daily_dict[daily_timestamp])
        else:
            fullDict[daily_timestamp] = [current_daily_dict[daily_timestamp]]
    

<Response [200]>


In [3]:

# Write to CSV file
with open(f'data/ge/{csvName}.csv', mode='w', newline='') as GE_data:
    GE_writer = csv.writer(GE_data, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for daily_timestamp in fullDict:
        new_array = [daily_timestamp]
        new_array.extend(fullDict[daily_timestamp])
        GE_writer.writerow(new_array)